In [25]:
import requests
import json
import csv
import time
import pandas as pd
import numpy as np
import copy
from typing import List, Dict, Any

# --- API 與實驗配置 ---
API_BASE_URL = "http://127.0.0.1:8000"
CAGE_CONFIG = {
    "id": "C001",
    "dimensions": [120.0, 100.0, 180.0],
    "weight_limit": 800.0
}
LOOKAHEAD_DEPTH = 3
TEMP_AREA_CAPACITY = 3
MCTS_SIMULATIONS = 50 # 固定 MCTS 模擬次數

# 定義我們要測試的所有演算法組合
ALGORITHM_COMBINATIONS = [
    {"strategy": "cp", "algorithm": "heuristics"},
    {"strategy": "cp", "algorithm": "mcts", "num_simu": MCTS_SIMULATIONS},
    {"strategy": "ems", "algorithm": "heuristics"}, # 如果您有 EMS，取消註解
    {"strategy": "ems", "algorithm": "mcts", "num_simu": MCTS_SIMULATIONS},
]

print("✅ 配置完畢，準備開始實驗。")

✅ 配置完畢，準備開始實驗。


In [26]:
def generate_dataset(num_items: int, seed: int):
    """
    產生一組可重複的貨物資料，格式為符合 API 要求的字典列表。
    """
    rng = np.random.RandomState(seed)
    dataset = []
    for i in range(num_items):
        dims = (
            rng.uniform(10, 60),
            rng.uniform(10, 60),
            rng.uniform(10, 60)
        )
        volume = dims[0] * dims[1] * dims[2]
        weight = (volume / 1000) * rng.uniform(0.5, 1.5)
        
        item_dict = {
            "id": i,
            "base_dimensions": dims,
            "weight": weight,
            "allowed_rotations": list(range(6)), 
            "is_fragile": False
        }
        dataset.append(item_dict)
    return dataset

print("✅ 資料集產生器定義完畢。")

✅ 資料集產生器定義完畢。


In [27]:
def run_api_experiment(algo_config: Dict, dataset: List[Dict]) -> Dict[str, Any]:
    """
    透過 API 呼叫，使用指定的演算法和資料集，執行一次完整的裝箱模擬。
    """
    # 1. 啟動伺服器端的裝箱會話
    try:
        response = requests.post(f"{API_BASE_URL}/start_packing", json=CAGE_CONFIG)
        response.raise_for_status()
    except requests.exceptions.RequestException as e:
        print(f"  ❌ 錯誤：無法啟動裝箱會話: {e}")
        return {"error": str(e)}

    # 2. 初始化客戶端狀態
    conveyor_items = copy.deepcopy(dataset)
    temp_area = []
    start_time = time.time()
    
    # 3. 主迴圈
    while conveyor_items or temp_area:
        all_list = temp_area + conveyor_items
        candidates = all_list[:LOOKAHEAD_DEPTH+1]
        if not candidates:
            break
            
        # 準備 API payload
        api_payload = {
            **algo_config, # 展開演算法配置，例如 {"strategy": "cp", "algorithm": "mcts"}
            "candidate_items": candidates
        }
        
        # 呼叫 API 進行決策
        try:
            response = requests.post(f"{API_BASE_URL}/decide_next_move", json=api_payload, timeout=60)
            response.raise_for_status()
            response_data = response.json()
        except requests.exceptions.RequestException as e:
            print(f"  ❌ 錯誤：API 決策請求失敗: {e}")
            break

        # 根據 API 回應更新客戶端狀態
        if response_data.get('status') == 'success':
            decision = response_data['decision']
            selected_item_id = decision['item']['id']
            
            # 從傳送帶或暫存區移除物品
            removed = False
            for i, item in enumerate(temp_area):
                if item['id'] == selected_item_id:
                    temp_area.pop(i)
                    removed = True
                    break
            if not removed:
                for i, item in enumerate(conveyor_items):
                    if item['id'] == selected_item_id:
                        conveyor_items.pop(i)
                        break
        elif response_data.get('status') == 'no_move_possible':
            if conveyor_items and len(temp_area) < TEMP_AREA_CAPACITY:
                temp_area.append(conveyor_items.pop(0))
            else:
                break # 無法再處理，結束模擬
        else:
            print(f"  ❌ 錯誤：收到未知的 API 回應: {response_data}")
            break
    
    end_time = time.time()

    # 4. 從伺服器獲取最終籠車狀態並計算指標
    try:
        response = requests.get(f"{API_BASE_URL}/get_cage_state")
        response.raise_for_status()
        final_cage_state = response.json().get('cage_state')
        
        if final_cage_state:
            packed_items = final_cage_state.get('packed_items', [])
            total_item_volume = 0
            for item in packed_items:
                # 加上 3cm 誤差來計算體積，與伺服器端保持一致
                dims = [d + 3.0 for d in item['base_dimensions']]
                total_item_volume += dims[0] * dims[1] * dims[2]
            
            cage_volume = CAGE_CONFIG['dimensions'][0] * CAGE_CONFIG['dimensions'][1] * CAGE_CONFIG['dimensions'][2]
            utilization = (total_item_volume / cage_volume) * 100 if cage_volume > 0 else 0
            
            return {
                "utilization": utilization,
                "item_count": len(packed_items),
                "time_taken": end_time - start_time,
                "final_weight": final_cage_state.get('current_weight', 0)
            }
    except requests.exceptions.RequestException as e:
        print(f"  ❌ 錯誤：無法獲取最終籠車狀態: {e}")

    return {"error": "無法獲取最終狀態"}

print("✅ API 實驗執行器函式定義完畢。")

✅ API 實驗執行器函式定義完畢。


In [ ]:
def csvtojson(csvfile_path: str) -> list[dict[str, Any]]:
    """
    從 CSV 檔案讀取貨物資料，並轉換為符合 API 要求的字典列表。
    """
    dataset = []
    with open(csvfile_path, mode='r', newline='') as csvfile:
        reader = csv.DictReader(csvfile)
        for row in reader:
            item_dict = {
                "id": int(row['id']),
                "base_dimensions": row['base_dimensions'].strip('()" '),
                "weight": float(row['weight']),
                "allowed_rotations": row['allowed_rotations'].strip('[]"\' '),
                "is_fragile": (row['is_fragile'].lower() == 'true')
            }
            dataset.append(item_dict)
    return dataset

# 使用既有資料集
ALGORITHMS = ALGORITHM_COMBINATIONS
REPETITIONS = 20 # 每個實驗重複 20 次
NUM_ITEMS_PER_DATASET = 40 # 每個資料集包含 40 個物品

all_results = []
total_runs = 5 * len(ALGORITHMS) * REPETITIONS
run_counter = 0

print(f"🚀 即將開始實驗，總共需要執行 {total_runs} 次模擬...")

for i in range(5):
    # 每個種子只產生一次資料集，以確保在同一個資料集上比較不同演算法
    csvflie_path = f"./cases/conveyor_items_{i}.csv"
    dataset = csvtojson(csvflie_path)
    print(f"\n--- 正在處理資料集  ---")
    
    for algo_key in ALGORITHMS:
        print(f"  > 正在運行演算法: {algo_key}")
        
        for rep in range(REPETITIONS):
            run_counter += 1
            print(f"    - 第 {rep + 1}/{REPETITIONS} 次重複... ({run_counter}/{total_runs})", end='\r')
            
            result_metrics = run_api_experiment(algo_key, dataset)
            
            all_results.append({
                # 'seed': seed,
                'algorithm': algo_key,
                'repetition': rep,
                'utilization': result_metrics['utilization'],
                'item_count': result_metrics['item_count'],
                'time_taken': result_metrics['time_taken'],
                'final_weight': result_metrics['final_weight']
            })
        print("\n    - 完成。")

# --- 將結果轉換為 DataFrame 並儲存 ---
results_df = pd.DataFrame(all_results)
results_df.to_csv('experiment_results.csv', index=False)

print("\n\n✅ 所有實驗執行完畢！結果已儲存至 'experiment_results.csv'")
display(results_df.head())

🚀 即將開始實驗，總共需要執行 400 次模擬...

--- 正在處理資料集  ---
  > 正在運行演算法: {'strategy': 'cp', 'algorithm': 'heuristics'}
  ❌ 錯誤：API 決策請求失敗: 422 Client Error: Unprocessable Entity for url: http://127.0.0.1:8000/decide_next_move
  ❌ 錯誤：API 決策請求失敗: 422 Client Error: Unprocessable Entity for url: http://127.0.0.1:8000/decide_next_move
  ❌ 錯誤：API 決策請求失敗: 422 Client Error: Unprocessable Entity for url: http://127.0.0.1:8000/decide_next_move
  ❌ 錯誤：API 決策請求失敗: 422 Client Error: Unprocessable Entity for url: http://127.0.0.1:8000/decide_next_move
  ❌ 錯誤：API 決策請求失敗: 422 Client Error: Unprocessable Entity for url: http://127.0.0.1:8000/decide_next_move
  ❌ 錯誤：API 決策請求失敗: 422 Client Error: Unprocessable Entity for url: http://127.0.0.1:8000/decide_next_move
  ❌ 錯誤：API 決策請求失敗: 422 Client Error: Unprocessable Entity for url: http://127.0.0.1:8000/decide_next_move
  ❌ 錯誤：API 決策請求失敗: 422 Client Error: Unprocessable Entity for url: http://127.0.0.1:8000/decide_next_move
  ❌ 錯誤：API 決策請求失敗: 422 Client Error: Unprocessab

ValueError: invalid literal for int() with base 10: 'False'

In [31]:
for i in range(5):
    # 每個種子只產生一次資料集，以確保在同一個資料集上比較不同演算法
    csvflie_path = f"./cases/conveyor_items_{i}.csv"
    dataset = csvtojson(csvflie_path)
    print(f"finished dataset {i}")

finished dataset 0


ValueError: invalid literal for int() with base 10: 'False'

In [ ]:
# --- 主實驗迴圈 ---
SEEDS = [0, 1, 2, 3, 4] # 5 個不同的資料集
ALGORITHMS = ALGORITHM_COMBINATIONS 
REPETITIONS = 20 # 每個實驗重複 20 次
NUM_ITEMS_PER_DATASET = 40 # 每個資料集包含 40 個物品

all_results = []
total_runs = len(SEEDS) * len(ALGORITHMS) * REPETITIONS
run_counter = 0

print(f"🚀 即將開始實驗，總共需要執行 {total_runs} 次模擬...")

for seed in SEEDS:
    # 每個種子只產生一次資料集，以確保在同一個資料集上比較不同演算法
    dataset = generate_dataset(num_items=NUM_ITEMS_PER_DATASET, seed=seed)
    print(f"\n--- 正在處理資料集 Seed={seed} ---")
    
    for algo_key in ALGORITHMS:
        print(f"  > 正在運行演算法: {algo_key}")
        
        for rep in range(REPETITIONS):
            run_counter += 1
            print(f"    - 第 {rep + 1}/{REPETITIONS} 次重複... ({run_counter}/{total_runs})", end='\r')
            
            result_metrics = run_api_experiment(algo_key, dataset)
            
            all_results.append({
                'seed': seed,
                'algorithm': algo_key,
                'repetition': rep,
                'utilization': result_metrics['utilization'],
                'item_count': result_metrics['item_count'],
                'time_taken': result_metrics['time_taken'],
                'final_weight': result_metrics['final_weight']
            })
        print("\n    - 完成。")

# --- 將結果轉換為 DataFrame 並儲存 ---
results_df = pd.DataFrame(all_results)
results_df.to_csv('experiment_results.csv', index=False)

print("\n\n✅ 所有實驗執行完畢！結果已儲存至 'experiment_results.csv'")
display(results_df.head())

🚀 即將開始實驗，總共需要執行 400 次模擬...

--- 正在處理資料集 Seed=0 ---
  > 正在運行演算法: {'strategy': 'cp', 'algorithm': 'heuristics'}
    - 第 20/20 次重複... (20/400)
    - 完成。
  > 正在運行演算法: {'strategy': 'cp', 'algorithm': 'mcts', 'num_simu': 50}
    - 第 20/20 次重複... (40/400)
    - 完成。
  > 正在運行演算法: {'strategy': 'ems', 'algorithm': 'heuristics'}


KeyboardInterrupt: 

In [ ]:
# --- 計算每個演算法的平均性能和穩定性 ---
# 按演算法分組，計算均值和標準差
summary_stats = results_df.groupby('algorithm').agg(
    avg_utilization=('utilization', 'mean'),
    std_utilization=('utilization', 'std'),
    avg_item_count=('item_count', 'mean'),
    avg_time=('time_taken', 'mean')
).reset_index().sort_values(by='avg_utilization', ascending=False)

print("--- 實驗結果摘要 (平均值與標準差) ---")
display(summary_stats)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
# --- 繪製比較圖表 ---

# 設定圖表風格
sns.set_theme(style="whitegrid")

# 1. 比較平均空間利用率
plt.figure(figsize=(12, 7))
ax1 = sns.barplot(data=summary_stats, x='algorithm', y='avg_utilization', palette='viridis')
ax1.set_title('各演算法平均空間利用率比較', fontsize=16)
ax1.set_ylabel('平均空間利用率 (%)', fontsize=12)
ax1.set_xlabel('演算法', fontsize=12)
# 在長條圖上顯示數值
for p in ax1.patches:
    ax1.annotate(f"{p.get_height():.2f}%", (p.get_x() + p.get_width() / 2., p.get_height()),
                 ha='center', va='center', fontsize=11, color='black', xytext=(0, 5),
                 textcoords='offset points')
plt.xticks(rotation=15)
plt.show()


# 2. 比較穩定性 (使用箱形圖)
plt.figure(figsize=(12, 7))
ax2 = sns.boxplot(data=results_df, x='algorithm', y='utilization', palette='coolwarm')
ax2.set_title('各演算法空間利用率穩定性比較 (箱形圖)', fontsize=16)
ax2.set_ylabel('空間利用率 (%)', fontsize=12)
ax2.set_xlabel('演算法', fontsize=12)
plt.xticks(rotation=15)
plt.show()

# 3. 比較平均執行時間
plt.figure(figsize=(12, 7))
ax3 = sns.barplot(data=summary_stats, x='algorithm', y='avg_time', palette='plasma')
ax3.set_title('各演算法平均執行時間比較', fontsize=16)
ax3.set_ylabel('平均執行時間 (秒)', fontsize=12)
ax3.set_xlabel('演算法', fontsize=12)
for p in ax3.patches:
    ax3.annotate(f"{p.get_height():.2f}s", (p.get_x() + p.get_width() / 2., p.get_height()),
                 ha='center', va='center', fontsize=11, color='black', xytext=(0, 5),
                 textcoords='offset points')
plt.xticks(rotation=15)
plt.show()